In [55]:
from __future__ import print_function


import numpy as np
import theano
import theano.tensor as T
import lasagne
import urllib2

In [56]:
#Lasagne Seed for Reproducibility
lasagne.random.set_rng(np.random.RandomState(1))

# Sequence Length
SEQ_LENGTH = 64

# Number of units in the two hidden (LSTM) layers
N_HIDDEN = 512

# Optimization learning rate
LEARNING_RATE = .01

# All gradients above this will be clipped
GRAD_CLIP = 100

# How often should we check the output?
PRINT_FREQ = 1

# Number of epochs to train the net
NUM_EPOCHS = 200

# Batch Size
BATCH_SIZE = 512

In [57]:
files = ["Data/irish_music.txt", "Data/swedish_music.txt","Data/french_music.txt"]
context = ["irish","swedish","french"]
in_text=[0]*len(files)
for i in range(0,len(files)) :
    try:
        #You can also use your own file
        #The file must be a simple text file.
        #Simply edit the file name below and uncomment the line.
        #in_text = open('data.txt').read() #nietzsche text
        in_text[i] = open(files[i], 'r').read() #Music file
        in_text[i]= in_text[i].replace('  ','')
        in_text[i]= in_text[i].replace('\r','')
        in_text[i]=''.join([j if ord(j) < 128 else ' ' for j in in_text[i]])
        in_text[i] = in_text[i].decode("utf-8-sig").encode("utf-8")
    except Exception as e:
        print(e)
        print("Please verify the location of the input file/URL.")
        print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
        raise IOError('Unable to Read Text')

In [58]:
#generation_phrase = "g2B2|d2d2 d2e2|d2cB g2fg|a2A2 A2\nGA|B2B2 cBAG|A2B2 g2fg|a2gf g2fe|d2B2 B2:|\n<end>\n<start>\n"
#generation_phrase="The quick brown fox jumps"
generation_phrases=[]
generation_phrases.append("a2 ba ge | d2 ed cB | c2 cd Bc | A4 AB |\ncB cd ef | g4 fe | d2 dB AG | E4 Bd |\ne2 ge dB | A2 B/c/d D2 | E2 GA/B/ A>G | G4 || \n<end>\n<start>\n")
generation_phrases.append("|:A2df a2a2 g4|A2^ce gage f3f|fafd ege^c d3e|f2ed ^c2ec A4|\nA2df a2a2 g4|A2^ce gage f3f|fafd ege^c d3e|f2ed ^c2ec d4:|\n<end>\n<start>\n")
generation_phrases.append("g2gb g2gb g2gb | f2fa f2fa f2fa | f2ed ^cde^c A4 :|\n|: A2d2 ^cde^c d4 | A2f2 efge f4 | A2d2 ^cde^c d4 |\nA2cA G2BG A4 | A2cA G2BG F2AF | EFE^C D8 :| \n<end>\n<start>\n")

In [59]:
a=set()
for text in in_text:
        a = a|set(text)
chars = list(a)
chars.sort()
data_size, vocab_size = sum(len(text) for text in in_text), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
vocab_size = vocab_size+3
n = len(chars)
char_to_ix['irish']=n
char_to_ix['swedish']=n+1
char_to_ix['french']=n+2
ix_to_char[n]='irish'
ix_to_char[n+1]='swedish' 
ix_to_char[n+2]='french' 

In [60]:
def gen_data(p, data,context, batch_size = BATCH_SIZE, return_target=True):
    '''
    This function produces a semi-redundant batch of training samples from the location 'p' in the provided string (data).
    For instance, assuming SEQ_LENGTH = 5 and p=0, the function would create batches of 
    5 characters of the string (starting from the 0th character and stepping by 1 for each semi-redundant batch)
    as the input and the next character as the target.
    To make this clear, let us look at a concrete example. Assume that SEQ_LENGTH = 5, p = 0 and BATCH_SIZE = 2
    If the input string was "The quick brown fox jumps over the lazy dog.",
    For the first data point,
    x (the inputs to the neural network) would correspond to the encoding of 'T','h','e',' ','q'
    y (the targets of the neural network) would be the encoding of 'u'
    For the second point,
    x (the inputs to the neural network) would correspond to the encoding of 'h','e',' ','q', 'u'
    y (the targets of the neural network) would be the encoding of 'i'
    The data points are then stacked (into a three-dimensional tensor of size (batch_size,SEQ_LENGTH,vocab_size))
    and returned. 
    Notice that there is overlap of characters between the batches (hence the name, semi-redundant batch).
    '''
    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)

    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
            x[n,i,char_to_ix[context]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')


In [61]:
import pickle
import numpy
from itertools import count

def main(num_epochs=NUM_EPOCHS):
    print("Building network ...")
    counter = count()
   
    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, SEQ_LENGTH, num_features)

    l_in = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

    # We now build the LSTM layer which takes l_in as the input layer
    # We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 

    l_forward_1 = lasagne.layers.LSTMLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

    l_forward_1_drop = lasagne.layers.DropoutLayer(l_forward_1, p=0.1)
    
    l_forward_2 = lasagne.layers.LSTMLayer(
        l_forward_1_drop, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

    # The l_forward layer creates an output of dimension (batch_size, SEQ_LENGTH, N_HIDDEN)
    # Since we are only interested in the final prediction, we isolate that quantity and feed it to the next layer. 
    # The output of the sliced layer will then be of size (batch_size, N_HIDDEN)
    l_forward_slice = lasagne.layers.SliceLayer(l_forward_2, -1, 1)

    # The sliced output is then passed through the softmax nonlinearity to create probability distribution of the prediction
    # The output of this stage is (batch_size, vocab_size)
    l_out = lasagne.layers.DenseLayer(l_forward_slice, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)

    # Theano tensor for the targets
    target_values = T.ivector('target_output')
    
    # lasagne.layers.get_output produces a variable for the output of the net
    network_output = lasagne.layers.get_output(l_out)

    # The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
    cost = T.nnet.categorical_crossentropy(network_output,target_values).mean()

    # Retrieve all parameters from the network
    params = pickle.load(open("params_context_country_3","rb"))
    lasagne.layers.set_all_param_values(l_out,params)
    all_params = lasagne.layers.get_all_params(l_out,trainable=True)

    # Compute AdaGrad updates for training
    print("Computing updates ...")
    updates = lasagne.updates.adagrad(cost, all_params, LEARNING_RATE)

    # Theano functions for training and computing cost
    print("Compiling functions ...")
    train = theano.function([l_in.input_var, target_values], cost, updates=updates, allow_input_downcast=True)
    compute_cost = theano.function([l_in.input_var, target_values], cost, allow_input_downcast=True)

    # In order to generate text from the network, we need the probability distribution of the next character given
    # the state of the network and the input (a seed).
    # In order to produce the probability distribution of the prediction, we compile a function called probs. 
    
    probs = theano.function([l_in.input_var],network_output,allow_input_downcast=True)

    # The next function generates text given a phrase of length at least SEQ_LENGTH.
    # The phrase is set using the variable generation_phrase.
    # The optional input "N" is used to set the number of characters of text to predict. 

    visualize = theano.function([l_in.input_var], lasagne.layers.get_output( l_forward_slice), allow_input_downcast=True)
    
    def try_it_out(context, generation_phrase, N=600):
        '''
        This function uses the user-provided string "generation_phrase" and current state of the RNN generate text.
        The function works in three steps:
        1. It converts the string set in "generation_phrase" (which must be over SEQ_LENGTH characters long) 
           to encoded format. We use the gen_data function for this. By providing the string and asking for a single batch,
           we are converting the first SEQ_LENGTH characters into encoded form. 
        2. We then use the LSTM to predict the next character and store it in a (dynamic) list sample_ix. This is done by using the 'probs'
           function which was compiled above. Simply put, given the output, we compute the probabilities of the target and pick the one 
           with the highest predicted probability. 
        3. Once this character has been predicted, we construct a new sequence using all but first characters of the 
           provided string and the predicted character. This sequence is then used to generate yet another character.
           This process continues for "N" characters. 
        To make this clear, let us again look at a concrete example. 
        Assume that SEQ_LENGTH = 5 and generation_phrase = "The quick brown fox jumps". 
        We initially encode the first 5 characters ('T','h','e',' ','q'). The next character is then predicted (as explained in step 2). 
        Assume that this character was 'J'. We then construct a new sequence using the last 4 (=SEQ_LENGTH-1) characters of the previous
        sequence ('h','e',' ','q') , and the predicted letter 'J'. This new sequence is then used to compute the next character and 
        the process continues.
        '''

        assert(len(generation_phrase)>=SEQ_LENGTH)
        sample_ix = []
        x,_ = gen_data(len(generation_phrase)-SEQ_LENGTH,generation_phrase,context,1,0)
        vis=[]
        inputs=""+generation_phrase[len(generation_phrase)-1]
        for i in range(N):
            # Pick the character that got assigned the highest probability
            ix = np.argmax(probs(x).ravel())
            op=visualize(x)
            vis.append([op[0,kk] for kk in range(512)])
            inputs+=ix_to_char[ix]
            # Alternatively, to sample from the distribution instead:
            # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
            sample_ix.append(ix)
            x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
            x[:,SEQ_LENGTH-1,:] = 0
            x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 
        vis=numpy.array(vis)
        index=str(next(counter))
        pickle.dump(vis,open("visualize/output"+index,'wb'))
        fname=open('visualize/input'+index+'.txt','w')
        fname.write(inputs)
        fname.close()

        random_snippet = ''.join(ix_to_char[ix] for ix in sample_ix)    
        print("----\n %s \n %s \n----" % (context,random_snippet))
        fname=open('Output/country_lstm_output_context_dropout_'+context+'.txt','a')
        fname.write("----\n<start>\n%s\n----" % (random_snippet))
        fname.close()
    
    print("Training ...")
    #print("Seed used for text generation is: " + generation_phrase)
    p = 0
    try:
        pos=0
        positions=[0]*len(files)
        for it in xrange(data_size * num_epochs / BATCH_SIZE):
            try_it_out(context[0],generation_phrases[0],N=600)
            try_it_out(context[1],generation_phrases[1],N=600)
            try_it_out(context[2],generation_phrases[2],N=600)# Generate text using the p^th character as the start. 
            
            avg_cost = 0;
            for _ in range(PRINT_FREQ):
                p=positions[pos]
                x,y = gen_data(p,in_text[pos],context[pos])
                
                #print(p)
                p += SEQ_LENGTH + BATCH_SIZE - 1 
                if(p+BATCH_SIZE+SEQ_LENGTH >= len(in_text[pos])):
                    print('Carriage Return')
                    p = 0;
                
                #print("context is ",context[pos],"positions is ", p)
                avg_cost += train(x, y)
                positions[pos]=p
                pos = (pos+1)%len(files)
            all_param = lasagne.layers.get_all_param_values(l_out)
            #  pickle.dump(all_param,open("params_context_country_playlist",'wb'))
            print("Epoch {} average loss = {}".format(it*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))
            fname=open('Output/country_lstm_loss_context_dropout_web.txt','a')
            fname.write("\n{},{}".format(it*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))
            fname.close()
                
    except Exception as e:
        print(e)

In [62]:
main()

Building network ...
Computing updates ...
Compiling functions ...
Training ...
----
 irish 
 X:22
T:Bourneel Bollon Boung
R:schontis
C:Trad Polla de cantes tmer (ilast Simon (1670-1933))
H:Antrotander in thaettess tute.
Z:iXXX-27i
Z:id:hn-schontiis-2
M:C|
K:Dm
A2 A>A B>cA|GE B>c d>e|fe ff/e/|df f>f|d>f fd c>d|Bc Bc|df gf ed|(3fec d2:|
<end>
<start>
X:12
T:La Reine da Laithi
R:cortlan
C:Tumalis tre ond
R:sl\"angeolska
C:Trad (hllsding, (1815-1980)
H:Joh brain in "umorie Jn'an Sellem the mert Frorch
R:slig\"ondes
Z:id:hn-sohk2-ie-s2-
M:C|
K:D
A2 |D FA dc AF|G3 E2 E2|cA B2 A2|c2 A2 cc|BA A2 G2|A4 A2|
B2 Bc B2|dc BA d2|e2 f2 fe|fe d2 f2|
e2 de fa|f2 d2 fe|de d2 ef|g2 fd fd|ef ed ^cd|1 B2  
----
----
 swedish 
 X:10
T:Locarilla (1922))
O:France
Z:ho-schottin
C:Martin La Lincsouvi
M:6/8
L:1/8
K:G
ef|g2 de fe|de ed Bc|d2 de fe|df gf ee|df ef gf|ed ef ge|
fe fe fe|dc BA G/A/F/A/|G2 GB GA|B2 A2 :|
<end>
<start>
X:10
T:La bainain
C:Palar Meligo
O:Franca
Z:id:hn-schottis-29
M:C|
L:1/8
K:G
e2d2 d

KeyboardInterrupt: 